In [1]:
from sshtunnel import SSHTunnelForwarder
import pymysql
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)

In [2]:
first_start_time='2024-06-26 06:00:00'
start_time='2024-07-11 00:00:00'
end_time='2024-07-17 23:59:59'

In [3]:
#  菲律宾ssh端口转发
tunnel = SSHTunnelForwarder(
    ('161.117.0.173', 22),
    ssh_username='liufengfang',
    ssh_password='liufengfang',
    ssh_pkey = r'C:\Users\Administrator\.ssh\id_rsa',
    remote_bind_address=('192.168.59.29',9030),
    local_bind_address=('127.0.0.1', 0))
tunnel.start()

In [4]:
# 连接菲律宾数仓
connection = pymysql.connect(
    host='127.0.0.1',
    port=tunnel.local_bind_port,
    user='u_liufengfang',
    password='UFCi3gIjo7qm',
    database='fox_ods'
)

In [5]:
# 业绩数据查询
query_sql_performance='''SELECT
	a.`业务组`,
	a.`主管`,
	a.`组长`,
	a.`催员ID`,
	COUNT(DISTINCT CONCAT(a.`催员ID`,IF(a.分案ID序列 = 1 AND a.`是否计入新案分案本金` = 1,a.`债务人ID`,NULL))) `新案分案个数`,
	COUNT(DISTINCT CONCAT(a.`催员ID`,IF(a.分案ID序列 = 1 AND a.`是否计入分案本金` = 1,a.`债务人ID`,NULL))) `分案个数`,
	SUM(IF(a.`分案ID序列` = 1 AND a.`是否计入新案分案本金` = 1,a.`分案本金`,0)) `新案分案本金`,
	SUM(IF(a.`分案ID序列` = 1 AND a.`是否计入分案本金` = 1,a.`分案本金`,0)) `总分案本金`,
	SUM(IF(a.`是否计入实收` = 1,a.`期次表催回总金额`,0)) `总实收`,
	SUM(IF(a.`分案ID序列` = 1 AND a.`是否计入新案分案本金` = 1 AND (DATE(a.`分案时间`) = DATE(a.`回款表回款时间`)),a.`分案本金`,0)) `新案D1回款的分案本金`,
	SUM(IF(a.`是否计入新案回款本金` = 1 AND (DATE(a.`分案时间`) = DATE(a.`回款表回款时间`)),a.`期次表催回本金`,0)) `新案D1回款本金`,
    SUM(IF(a.`是否计入新案回款本金` = 1,a.`期次表催回本金`,0))`新案回款本金`,
    SUM(IF(a.`是否计入回款本金` = 1,a.`期次表催回本金`,0))`回款本金`
FROM
	(SELECT
		uod.asset_group_name `业务组`,
		uod.manager_user_name `主管`,
		uod.leader_user_name `组长`,
		ml.assigned_sys_user_id `催员ID`,
		ml.mission_log_asset_id `资产ID`,
		asset.asset_item_number `资产编号`,
		ml.debtor_id `债务人ID`,
		ml.mission_log_id `分案id`,
		ml.assign_asset_late_days `分案时资产逾期天数`,
		ml.assign_principal_amount * 0.01 `分案本金`,
		ml.mission_log_create_at `分案时间`,
		unml.mission_log_create_at `撤案时间`,
		IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) `撤案理由`,
		(CASE cad.attendance_status_flag
		WHEN 1 THEN '上线'
		WHEN 3 THEN '下线'
		WHEN 4 THEN '短假'
		WHEN 5 THEN '长假'
		WHEN 6 THEN '矿工'
		WHEN 7 THEN '稽核下线'
		ELSE '其它状态'
		END) `分案时催员状态`,
		cad.attendance_status_flag `分案时催员状态编号`,
		(CASE cadcr.attendance_status_flag
		WHEN 1 THEN '上线'
		WHEN 3 THEN '下线'
		WHEN 4 THEN '短假'
		WHEN 5 THEN '长假'
		WHEN 6 THEN '矿工'
		WHEN 7 THEN '稽核下线'
		ELSE '其它状态'
		END) `回款时催员状态`,
		cadcr.attendance_status_flag `回款时催员状态编号`,
		IFNULL(cr.repaid_principal_amount * 0.01,0) `回款表回款本金`,
		IFNULL(cr.delay_amount * 0.01,0) `回款表展期费用`,
		cr.repay_date `回款表回款时间`,
		cr.create_at `回款表创建时间`,
		IFNULL(apr.delay_amount * 0.01,0) `期次表展期费用`,
		IFNULL(apr.repaid_principal_amount * 0.01,0) `期次表催回本金`,
		IFNULL(apr.repaid_fee_amount * 0.01,0) `期次表催回手续费`,
		IFNULL(apr.repaid_penalty_amount * 0.01,0) `期次表催回罚息`,
		IFNULL(apr.repaid_interest_amount * 0.01,0) `期次表催回利息`,
		IFNULL(apr.repaid_total_amount * 0.01,0) `期次表催回总金额`,
		cr.batch_num `还款批次号`,
		apr.repaid_period `还款期次`,
		ml.overdue_period `分案期次`,
		cr.overdue_flag `是否逾期`,
		ROW_NUMBER() OVER(PARTITION BY ml.mission_log_id) `分案ID序列`,
		
		
		-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 当天分案当天展期不计入
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) AND
		-- 因为撤案原因 当天撤案 并且 当天没有回款 不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'稽核异常撤案,风险上报转客维,稽核投诉倾向撤案,转客维,AUDIT_ABNORMAL_WITHDRAW_THE_CASE,RISK_REPORTING_TRANSFERRED_TO_CUSTOMER_SERVICE,AUDIT_COMPLAINT_TEND_WITHDRAWAL_CASE') 
			AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at) AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0))) `是否计入分案本金`,
		

	-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
			-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 当天分案当天展期不计入
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) AND
		-- 因为撤案原因 当天撤案 并且 当天没有回款 不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'稽核异常撤案,风险上报转客维,稽核投诉倾向撤案,转客维,AUDIT_ABNORMAL_WITHDRAW_THE_CASE,RISK_REPORTING_TRANSFERRED_TO_CUSTOMER_SERVICE,AUDIT_COMPLAINT_TEND_WITHDRAWAL_CASE') 
			AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at) AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) AND
			-- 各业务组新案判断
			(IF(	(ml.mission_group_name LIKE 'Pre%' AND ml.assign_asset_late_days = -2)
					OR (ml.mission_group_name LIKE 'A%' AND ml.assign_asset_late_days = 1)
					OR (ml.mission_group_name LIKE 'B%' AND ml.assign_asset_late_days IN (1,8))
					OR (ml.mission_group_name LIKE 'M2%' AND ml.assign_asset_late_days IN (1,31)),1,0))) `是否计入新案分案本金`,


		-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
			-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) 
	-- 长假、稽核下线当天回款不计入
			AND (NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),0) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
			-- 预提醒只统计当期回款本金，其他业务组统计逾期回款
		(IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0,1,
			IF(cr.overdue_flag = 1 AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0 ,1,0))) ) `是否计入回款本金`,
			
			
			-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
			-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) 
	-- 长假、稽核下线当天回款不计入
			AND (NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),0) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
			-- 预提醒只统计当期回款本金，其他业务组统计逾期回款
		(IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0,1,
			IF(cr.overdue_flag = 1 AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0 ,1,0))) AND
			-- 各业务组新案判断
			(IF(	(ml.mission_group_name LIKE 'Pre%' AND ml.assign_asset_late_days = -2)
					OR (ml.mission_group_name LIKE 'A%' AND ml.assign_asset_late_days = 1)
					OR (ml.mission_group_name LIKE 'B%' AND ml.assign_asset_late_days IN (1,8))
					OR (ml.mission_group_name LIKE 'M2%' AND ml.assign_asset_late_days IN (1,31)),1,0))) `是否计入新案回款本金`,
			
			
				-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- 当天分案当天展期不计入(展期无回款本金，不用判断)
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 长假、稽核下线当天回款不计入
		 (NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),1) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
		 -- 展期金额不为空
		 (IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.delay_amount,0) <> 0,1,
				IF(LOCATE('PreRemind',ml.mission_group_name) = 0 AND IFNULL(apr.delay_amount,0) <> 0,1,0)))) `是否计入展期`,
				
				
		-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- 当天分案当天展期不计入(展期无回款本金，不用判断)
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 长假、稽核下线当天回款不计入
		(NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),1) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
		-- 展期金额不为空
		(IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.delay_amount,0) <> 0,1,
				IF(LOCATE('PreRemind',ml.mission_group_name) = 0 AND IFNULL(apr.delay_amount,0) <> 0,1,0))) AND
		-- 各业务组新案判断
		(IF((ml.mission_group_name LIKE 'Pre%' AND ml.assign_asset_late_days = -2)
			OR (ml.mission_group_name LIKE 'A%' AND ml.assign_asset_late_days = 1)
			OR (ml.mission_group_name LIKE 'B%' AND ml.assign_asset_late_days IN (1,8))
			OR (ml.mission_group_name LIKE 'M2%' AND ml.assign_asset_late_days IN (1,31)),1,0))) `是否计入新案展期`,
	-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) 
	-- 长假、稽核下线当天回款不计入
			AND NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),0) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) `是否计入实收`
	FROM
		ods_fox_mission_log ml
		LEFT JOIN ods_fox_mission_log_unassign_reason mlur ON ml.mission_log_id = mlur.assign_mission_log_id
		LEFT JOIN ods_fox_mission_log unml ON mlur.mission_log_id = unml.mission_log_id 
		AND unml.mission_log_operator = 'unassign' 
		AND ( unml.mission_log_create_at >= '{0}' AND unml.mission_log_create_at <= '{1}' ) 
		LEFT JOIN ods_fox_collect_attendance_dtl cad ON ml.assigned_sys_user_id = cad.user_id
		AND DATE(ml.mission_log_create_at) = cad.work_day
		LEFT JOIN ods_fox_collect_recovery cr ON ml.mission_log_id = cr.mission_log_id
		AND ( cr.create_at >= '{0}' AND cr.create_at <= '{1}' ) 
		AND cr.batch_num IS NOT NULL
		LEFT JOIN ods_fox_asset_period_recovery apr ON cr.batch_num = apr.batch_num
		LEFT JOIN ods_fox_collect_attendance_dtl cadcr ON cr.sys_user_id = cadcr.user_id
		AND DATE(cr.create_at) = cadcr.work_day
		LEFT JOIN ods_fox_asset asset ON ml.mission_log_asset_id = asset.asset_id
		LEFT JOIN
		(SELECT SUBSTRING_INDEX( uo.asset_group_name, ',',- 1 ) `asset_group_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 2 ), ',', - 1 ) `manager_user_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 3 ), ',', - 1 ) `leader_user_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 4 ), ',', - 1 ) `组员`,
			uo.user_id,
			uo.pt_date
		FROM
			fox_dw.dwd_fox_user_organization_df uo
		WHERE
			(uo.pt_date BETWEEN '{0}' AND '{1}')) uod ON ml.assigned_sys_user_id = uod.user_id
		AND DATE(ml.mission_log_create_at) = DATE(uod.pt_date)
	WHERE
		ml.mission_log_operator = 'assign'
		AND (ml.mission_log_create_at >= '{0}' AND ml.mission_log_create_at <= '{1}')
		AND ml.mission_group_name NOT IN ('PHCC','IVR','WIZ-IVR','cultivate')) a
GROUP BY
	a.`业务组`,
	a.`主管`,
	a.`组长`,
	a.`催员ID`'''

In [6]:
# 上线日期出勤天数是否新人查询
query_sql_online_new='''SELECT
	`架构`.asset_group_name ,
	`架构`.manager_user_name,
	`架构`.leader_user_name,
	cad.user_id,
	cad.work_day,
	first_online.`上线日期`,
	cad.attendance_status,
	MAX(cad.work_day) OVER(PARTITION BY `架构`.asset_group_name,`架构`.manager_user_name,`架构`.leader_user_name,cad.user_id) `最近上线日期`,
	(DATE_ADD(first_online.`上线日期`,INTERVAL 29 DAY) > 
		MAX(cad.work_day) OVER(PARTITION BY `架构`.asset_group_name,`架构`.manager_user_name,`架构`.leader_user_name,cad.user_id)) `是否新人`
FROM
	ods_fox_collect_attendance_dtl cad
	LEFT JOIN
	(SELECT
			cad.user_id,
			IF(MIN( cad.work_day ) > DATE(MIN( od.create_at )),DATE(MIN( od.create_at )),MIN( cad.work_day ))`上线日期`
		FROM
			ods_fox_collect_attendance_dtl cad
			LEFT JOIN ods_fox_online_days od ON cad.user_id = od.user_id 
			AND od.online_day_flag = 1 
		WHERE
			cad.attendance_status = 1 
		GROUP BY
			cad.user_id) `first_online` ON cad.user_id = `first_online`.user_id
	LEFT JOIN
	(SELECT
			SUBSTRING_INDEX( uo.asset_group_name, ',',- 1 ) `asset_group_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 2 ), ',', - 1 ) `manager_user_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 3 ), ',', - 1 ) `leader_user_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 4 ), ',', - 1 ) `组员`,
			uo.user_id,
			uo.pt_date 
		FROM
			fox_dw.dwd_fox_user_organization_df uo 
		WHERE
			(uo.pt_date BETWEEN '{0}' AND '{1}')) `架构` ON cad.user_id = `架构`.user_id
		AND cad.work_day = `架构`.pt_date
WHERE
	cad.attendance_status = 1
	AND (cad.work_day BETWEEN '{0}' AND '{1}')'''

In [7]:
# 通话数据
query_sql_call = '''SELECT
	uo.asset_group_name `业务组`,
	uo.manager_user_name `主管`,
	uo.leader_user_name `组长`,
	che.dunner_id `催员ID`,
	COUNT(ch.call_at) `外呼次数`,
	COUNT(IF(ch.call_time >0 ,1,NULL)) `接通次数`,
	SUM(ch.call_time) `接通时长`,
	COUNT(DISTINCT che.debtor_id) `总外呼债务人`,
	COUNT(DISTINCT IF(che.debtor_relationship <> 'self',ch.enc_debtor_phone_number,NULL)) `三方总外呼号码数量`,
	COUNT(DISTINCT IF(che.debtor_relationship = 'self' AND ch.call_time > 0,che.debtor_id,NULL)) `本人接通总债务人`,
	COUNT(DISTINCT IF(che.debtor_relationship <> 'self' AND ch.call_time > 0,che.debtor_id,NULL)) `三方接通总债务人`,
	COUNT(DISTINCT IF(ch.call_time > 0,che.debtor_id,NULL)) `接通总债务人`
FROM
	ods_audit_call_history ch
	LEFT JOIN ods_audit_call_history_extend che ON ch.id = che.source_id
	LEFT JOIN
	(SELECT
			SUBSTRING_INDEX( uo.asset_group_name, ',',- 1 ) `asset_group_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 2 ), ',', - 1 ) `manager_user_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 3 ), ',', - 1 ) `leader_user_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 4 ), ',', - 1 ) `组员`,
			uo.user_id,
			uo.pt_date 
		FROM
			fox_dw.dwd_fox_user_organization_df uo 
		WHERE
			(uo.pt_date BETWEEN '{0}' AND '{1}')) `uo` ON che.dunner_id = uo.user_id
			AND DATE(ch.call_at) = uo.pt_date
WHERE
	(ch.call_at BETWEEN '{0}' AND '{1}')
	AND ch.call_channel = 1
	AND (che.dunner_id,che.debtor_id,che.asset_id) IN
	(SELECT DISTINCT
	a.`催员ID`,
	a.`债务人ID`,
	a.`资产ID`
FROM
(SELECT
		ml.assigned_sys_user_id `催员ID`,
		ml.mission_log_asset_id `资产ID`,
		asset.asset_item_number `资产编号`,
		ml.debtor_id `债务人ID`,
		ml.mission_log_id `分案id`,
		ml.assign_asset_late_days `分案时资产逾期天数`,
		ml.assign_principal_amount * 0.01 `分案本金`,
		ml.mission_log_create_at `分案时间`,
		unml.mission_log_create_at `撤案时间`,
		IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) `撤案理由`,
		(CASE cad.attendance_status_flag
		WHEN 1 THEN '上线'
		WHEN 3 THEN '下线'
		WHEN 4 THEN '短假'
		WHEN 5 THEN '长假'
		WHEN 6 THEN '矿工'
		WHEN 7 THEN '稽核下线'
		ELSE '其它状态'
		END) `分案时催员状态`,
		cad.attendance_status_flag `分案时催员状态编号`,
		(CASE cadcr.attendance_status_flag
		WHEN 1 THEN '上线'
		WHEN 3 THEN '下线'
		WHEN 4 THEN '短假'
		WHEN 5 THEN '长假'
		WHEN 6 THEN '矿工'
		WHEN 7 THEN '稽核下线'
		ELSE '其它状态'
		END) `回款时催员状态`,
		cadcr.attendance_status_flag `回款时催员状态编号`,
		IFNULL(cr.repaid_principal_amount * 0.01,0) `回款表回款本金`,
		IFNULL(cr.delay_amount * 0.01,0) `回款表展期费用`,
		cr.repay_date `回款表回款时间`,
		cr.create_at `回款表创建时间`,
		IFNULL(apr.delay_amount * 0.01,0) `期次表展期费用`,
		IFNULL(apr.repaid_principal_amount * 0.01,0) `期次表催回本金`,
		IFNULL(apr.repaid_fee_amount * 0.01,0) `期次表催回手续费`,
		IFNULL(apr.repaid_penalty_amount * 0.01,0) `期次表催回罚息`,
		IFNULL(apr.repaid_interest_amount * 0.01,0) `期次表催回利息`,
		IFNULL(apr.repaid_total_amount * 0.01,0) `期次表催回总金额`,
		cr.batch_num `还款批次号`,
		apr.repaid_period `还款期次`,
		ml.overdue_period `分案期次`,
		cr.overdue_flag `是否逾期`,
		ROW_NUMBER() OVER(PARTITION BY ml.mission_log_id) `分案ID序列`,
		
		
		-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 当天分案当天展期不计入
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) AND
		-- 因为撤案原因 当天撤案 并且 当天没有回款 不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'稽核异常撤案,风险上报转客维,稽核投诉倾向撤案,转客维,AUDIT_ABNORMAL_WITHDRAW_THE_CASE,RISK_REPORTING_TRANSFERRED_TO_CUSTOMER_SERVICE,AUDIT_COMPLAINT_TEND_WITHDRAWAL_CASE') 
			AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at) AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0))) `是否计入分案本金`,
		

	-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
			-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 当天分案当天展期不计入
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) AND
		-- 因为撤案原因 当天撤案 并且 当天没有回款 不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'稽核异常撤案,风险上报转客维,稽核投诉倾向撤案,转客维,AUDIT_ABNORMAL_WITHDRAW_THE_CASE,RISK_REPORTING_TRANSFERRED_TO_CUSTOMER_SERVICE,AUDIT_COMPLAINT_TEND_WITHDRAWAL_CASE') 
			AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at) AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) AND
			-- 各业务组新案判断
			(IF(	(ml.mission_group_name LIKE 'Pre%' AND ml.assign_asset_late_days = -2)
					OR (ml.mission_group_name LIKE 'A%' AND ml.assign_asset_late_days = 1)
					OR (ml.mission_group_name LIKE 'B%' AND ml.assign_asset_late_days IN (1,8))
					OR (ml.mission_group_name LIKE 'M2%' AND ml.assign_asset_late_days IN (1,31)),1,0))) `是否计入新案分案本金`,


		-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
			-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) 
	-- 长假、稽核下线当天回款不计入
			AND (NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),0) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
			-- 预提醒只统计当期回款本金，其他业务组统计逾期回款
		(IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0,1,
			IF(cr.overdue_flag = 1 AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0 ,1,0))) ) `是否计入回款本金`,
			
			
			-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
			-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) 
	-- 长假、稽核下线当天回款不计入
			AND (NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),0) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
			-- 预提醒只统计当期回款本金，其他业务组统计逾期回款
		(IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0,1,
			IF(cr.overdue_flag = 1 AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0 ,1,0))) AND
			-- 各业务组新案判断
			(IF(	(ml.mission_group_name LIKE 'Pre%' AND ml.assign_asset_late_days = -2)
					OR (ml.mission_group_name LIKE 'A%' AND ml.assign_asset_late_days = 1)
					OR (ml.mission_group_name LIKE 'B%' AND ml.assign_asset_late_days IN (1,8))
					OR (ml.mission_group_name LIKE 'M2%' AND ml.assign_asset_late_days IN (1,31)),1,0))) `是否计入新案回款本金`,
			
			
				-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- 当天分案当天展期不计入(展期无回款本金，不用判断)
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 长假、稽核下线当天回款不计入
		 (NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),1) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
		 -- 展期金额不为空
		 (IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.delay_amount,0) <> 0,1,
				IF(LOCATE('PreRemind',ml.mission_group_name) = 0 AND IFNULL(apr.delay_amount,0) <> 0,1,0)))) `是否计入展期`,
				
				
		-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- 当天分案当天展期不计入(展期无回款本金，不用判断)
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 长假、稽核下线当天回款不计入
		(NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),1) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
		-- 展期金额不为空
		(IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.delay_amount,0) <> 0,1,
				IF(LOCATE('PreRemind',ml.mission_group_name) = 0 AND IFNULL(apr.delay_amount,0) <> 0,1,0))) AND
		-- 各业务组新案判断
		(IF((ml.mission_group_name LIKE 'Pre%' AND ml.assign_asset_late_days = -2)
			OR (ml.mission_group_name LIKE 'A%' AND ml.assign_asset_late_days = 1)
			OR (ml.mission_group_name LIKE 'B%' AND ml.assign_asset_late_days IN (1,8))
			OR (ml.mission_group_name LIKE 'M2%' AND ml.assign_asset_late_days IN (1,31)),1,0))) `是否计入新案展期`,
	-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) 
	-- 长假、稽核下线当天回款不计入
			AND NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),0) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) `是否计入实收`
	FROM
		ods_fox_mission_log ml
		LEFT JOIN ods_fox_mission_log_unassign_reason mlur ON ml.mission_log_id = mlur.assign_mission_log_id
		LEFT JOIN ods_fox_mission_log unml ON mlur.mission_log_id = unml.mission_log_id 
		AND unml.mission_log_operator = 'unassign' 
		AND ( unml.mission_log_create_at >= '{0}' AND unml.mission_log_create_at <= '{1}' ) 
		LEFT JOIN ods_fox_collect_attendance_dtl cad ON ml.assigned_sys_user_id = cad.user_id
		AND DATE(ml.mission_log_create_at) = cad.work_day
		LEFT JOIN ods_fox_collect_recovery cr ON ml.mission_log_id = cr.mission_log_id
		AND ( cr.create_at >= '{0}' AND cr.create_at <= '{1}' ) 
		AND cr.batch_num IS NOT NULL
		LEFT JOIN ods_fox_asset_period_recovery apr ON cr.batch_num = apr.batch_num
		LEFT JOIN ods_fox_collect_attendance_dtl cadcr ON cr.sys_user_id = cadcr.user_id
		AND DATE(cr.create_at) = cadcr.work_day
		LEFT JOIN ods_fox_asset asset ON ml.mission_log_asset_id = asset.asset_id
	WHERE
		ml.mission_log_operator = 'assign'
		AND (ml.mission_log_create_at >= '{0}' AND ml.mission_log_create_at <= '{1}')
		AND ml.mission_group_name NOT IN ('PHCC','IVR','WIZ-IVR','cultivate')) a
	WHERE
		a.`是否计入分案本金` = 1 and a.分案ID序列 = 1)
GROUP BY
	uo.asset_group_name,
	uo.manager_user_name,
	uo.leader_user_name,
	che.dunner_id'''

In [8]:
# 短信发送量
query_sql_sms = '''SELECT
	uo.asset_group_name `业务组`,
	uo.manager_user_name `主管`,
	uo.leader_user_name `组长`,
	she.dunner_id `催员ID`,
	COUNT(sh.request_id) `短信发送数量`
FROM
	ods_audit_sms_history sh
	LEFT JOIN ods_audit_sms_history_extend she ON sh.id = she.source_id
	LEFT JOIN 
	(SELECT
			SUBSTRING_INDEX( uo.asset_group_name, ',',- 1 ) `asset_group_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 2 ), ',', - 1 ) `manager_user_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 3 ), ',', - 1 ) `leader_user_name`,
			SUBSTRING_INDEX( SUBSTRING_INDEX( uo.parent_user_names, ',', 4 ), ',', - 1 ) `组员`,
			uo.user_id,
			uo.pt_date
		FROM
			fox_dw.dwd_fox_user_organization_df uo 
		WHERE
			(uo.pt_date BETWEEN '{0}' AND '{1}')) `uo` ON she.dunner_id = uo.user_id
			AND DATE(she.create_at) = uo.pt_date
WHERE
	(sh.sms_at BETWEEN '{0}' AND '{1}')
	AND sh.sms_channel = 1
	AND (she.dunner_id,she.debtor_id,she.asset_id) IN
	(SELECT DISTINCT
	a.`催员ID`,
	a.`债务人ID`,
	a.`资产ID`
FROM
(SELECT
		ml.assigned_sys_user_id `催员ID`,
		ml.mission_log_asset_id `资产ID`,
		asset.asset_item_number `资产编号`,
		ml.debtor_id `债务人ID`,
		ml.mission_log_id `分案id`,
		ml.assign_asset_late_days `分案时资产逾期天数`,
		ml.assign_principal_amount * 0.01 `分案本金`,
		ml.mission_log_create_at `分案时间`,
		unml.mission_log_create_at `撤案时间`,
		IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) `撤案理由`,
		(CASE cad.attendance_status_flag
		WHEN 1 THEN '上线'
		WHEN 3 THEN '下线'
		WHEN 4 THEN '短假'
		WHEN 5 THEN '长假'
		WHEN 6 THEN '矿工'
		WHEN 7 THEN '稽核下线'
		ELSE '其它状态'
		END) `分案时催员状态`,
		cad.attendance_status_flag `分案时催员状态编号`,
		(CASE cadcr.attendance_status_flag
		WHEN 1 THEN '上线'
		WHEN 3 THEN '下线'
		WHEN 4 THEN '短假'
		WHEN 5 THEN '长假'
		WHEN 6 THEN '矿工'
		WHEN 7 THEN '稽核下线'
		ELSE '其它状态'
		END) `回款时催员状态`,
		cadcr.attendance_status_flag `回款时催员状态编号`,
		IFNULL(cr.repaid_principal_amount * 0.01,0) `回款表回款本金`,
		IFNULL(cr.delay_amount * 0.01,0) `回款表展期费用`,
		cr.repay_date `回款表回款时间`,
		cr.create_at `回款表创建时间`,
		IFNULL(apr.delay_amount * 0.01,0) `期次表展期费用`,
		IFNULL(apr.repaid_principal_amount * 0.01,0) `期次表催回本金`,
		IFNULL(apr.repaid_fee_amount * 0.01,0) `期次表催回手续费`,
		IFNULL(apr.repaid_penalty_amount * 0.01,0) `期次表催回罚息`,
		IFNULL(apr.repaid_interest_amount * 0.01,0) `期次表催回利息`,
		IFNULL(apr.repaid_total_amount * 0.01,0) `期次表催回总金额`,
		cr.batch_num `还款批次号`,
		apr.repaid_period `还款期次`,
		ml.overdue_period `分案期次`,
		cr.overdue_flag `是否逾期`,
		ROW_NUMBER() OVER(PARTITION BY ml.mission_log_id) `分案ID序列`,
		
		
		-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 当天分案当天展期不计入
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) AND
		-- 因为撤案原因 当天撤案 并且 当天没有回款 不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'稽核异常撤案,风险上报转客维,稽核投诉倾向撤案,转客维,AUDIT_ABNORMAL_WITHDRAW_THE_CASE,RISK_REPORTING_TRANSFERRED_TO_CUSTOMER_SERVICE,AUDIT_COMPLAINT_TEND_WITHDRAWAL_CASE') 
			AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at) AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0))) `是否计入分案本金`,
		

	-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
			-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 当天分案当天展期不计入
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) AND
		-- 因为撤案原因 当天撤案 并且 当天没有回款 不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'稽核异常撤案,风险上报转客维,稽核投诉倾向撤案,转客维,AUDIT_ABNORMAL_WITHDRAW_THE_CASE,RISK_REPORTING_TRANSFERRED_TO_CUSTOMER_SERVICE,AUDIT_COMPLAINT_TEND_WITHDRAWAL_CASE') 
			AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at) AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) AND
			-- 各业务组新案判断
			(IF(	(ml.mission_group_name LIKE 'Pre%' AND ml.assign_asset_late_days = -2)
					OR (ml.mission_group_name LIKE 'A%' AND ml.assign_asset_late_days = 1)
					OR (ml.mission_group_name LIKE 'B%' AND ml.assign_asset_late_days IN (1,8))
					OR (ml.mission_group_name LIKE 'M2%' AND ml.assign_asset_late_days IN (1,31)),1,0))) `是否计入新案分案本金`,


		-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
			-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) 
	-- 长假、稽核下线当天回款不计入
			AND (NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),0) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
			-- 预提醒只统计当期回款本金，其他业务组统计逾期回款
		(IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0,1,
			IF(cr.overdue_flag = 1 AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0 ,1,0))) ) `是否计入回款本金`,
			
			
			-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
			-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) 
	-- 长假、稽核下线当天回款不计入
			AND (NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),0) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
			-- 预提醒只统计当期回款本金，其他业务组统计逾期回款
		(IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0,1,
			IF(cr.overdue_flag = 1 AND IFNULL(apr.repaid_principal_amount * 0.01,0) <> 0 ,1,0))) AND
			-- 各业务组新案判断
			(IF(	(ml.mission_group_name LIKE 'Pre%' AND ml.assign_asset_late_days = -2)
					OR (ml.mission_group_name LIKE 'A%' AND ml.assign_asset_late_days = 1)
					OR (ml.mission_group_name LIKE 'B%' AND ml.assign_asset_late_days IN (1,8))
					OR (ml.mission_group_name LIKE 'M2%' AND ml.assign_asset_late_days IN (1,31)),1,0))) `是否计入新案回款本金`,
			
			
				-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- 当天分案当天展期不计入(展期无回款本金，不用判断)
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 长假、稽核下线当天回款不计入
		 (NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),1) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
		 -- 展期金额不为空
		 (IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.delay_amount,0) <> 0,1,
				IF(LOCATE('PreRemind',ml.mission_group_name) = 0 AND IFNULL(apr.delay_amount,0) <> 0,1,0)))) `是否计入展期`,
				
				
		-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- 当天分案当天展期不计入(展期无回款本金，不用判断)
		(NOT (IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason) = '展期' AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at))) AND
		-- 长假、稽核下线当天回款不计入
		(NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),1) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) AND
		-- 展期金额不为空
		(IF(LOCATE('PreRemind',ml.mission_group_name) AND ml.overdue_period = apr.repaid_period AND IFNULL(apr.delay_amount,0) <> 0,1,
				IF(LOCATE('PreRemind',ml.mission_group_name) = 0 AND IFNULL(apr.delay_amount,0) <> 0,1,0))) AND
		-- 各业务组新案判断
		(IF((ml.mission_group_name LIKE 'Pre%' AND ml.assign_asset_late_days = -2)
			OR (ml.mission_group_name LIKE 'A%' AND ml.assign_asset_late_days = 1)
			OR (ml.mission_group_name LIKE 'B%' AND ml.assign_asset_late_days IN (1,8))
			OR (ml.mission_group_name LIKE 'M2%' AND ml.assign_asset_late_days IN (1,31)),1,0))) `是否计入新案展期`,
	-- 分案时催员长假、稽核下线不计入
		((NOT IFNULL(FIND_IN_SET(cad.attendance_status_flag,'5,7'),0)) AND
		-- -- 分案时催员短假、旷工并且当天所有撤案不计入
		(NOT IF(IFNULL(FIND_IN_SET(cad.attendance_status_flag,'4,6'),0) AND DATE(ml.mission_log_create_at) = DATE(unml.mission_log_create_at),1,0)) AND
		-- 因为撤案原因不计入
		(NOT FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'分案不均,拨打受限组内互换案件,分案前已结清,恢复案件撤案,分案不均补案,存在逾期案件时撤d-2案件,UNEVEN_WITHDRAW_CASE,GROUP_EXCHANGE_WITHDRAWAL_CASE,RESUME_CASE_DISMISSAL,WITHDRAW_D_2_CASES_WHEN_THERE_ARE_OVERDUE_CASES
,SETTLED_BEFORE_DIVISION')) AND
		-- 因为撤案原因并且没有回款不计入
		(NOT (FIND_IN_SET(IF(ISNULL(unml.mission_log_create_at),'当前未撤案',mlur.mission_log_unassign_reason),'因不可抗力因素温和催收,正常撤案,冻结债务人,冻结,MODERATE_WITHDRAW_CASE,DISMISS_THE_CASE_NORMALLY,FREEZE_DEBTOR,FREEZE') AND IFNULL(apr.repaid_total_amount * 0.01,0) = 0)) 
	-- 长假、稽核下线当天回款不计入
			AND NOT (IFNULL(FIND_IN_SET(cadcr.attendance_status_flag,'5,7'),0) AND IFNULL(DATE(cr.create_at) = (cadcr.work_day),1))) `是否计入实收`
	FROM
		ods_fox_mission_log ml
		LEFT JOIN ods_fox_mission_log_unassign_reason mlur ON ml.mission_log_id = mlur.assign_mission_log_id
		LEFT JOIN ods_fox_mission_log unml ON mlur.mission_log_id = unml.mission_log_id 
		AND unml.mission_log_operator = 'unassign' 
		AND ( unml.mission_log_create_at >= '{0}' AND unml.mission_log_create_at <= '{1}' ) 
		LEFT JOIN ods_fox_collect_attendance_dtl cad ON ml.assigned_sys_user_id = cad.user_id
		AND DATE(ml.mission_log_create_at) = cad.work_day
		LEFT JOIN ods_fox_collect_recovery cr ON ml.mission_log_id = cr.mission_log_id
		AND ( cr.create_at >= '{0}' AND cr.create_at <= '{1}' ) 
		AND cr.batch_num IS NOT NULL
		LEFT JOIN ods_fox_asset_period_recovery apr ON cr.batch_num = apr.batch_num
		LEFT JOIN ods_fox_collect_attendance_dtl cadcr ON cr.sys_user_id = cadcr.user_id
		AND DATE(cr.create_at) = cadcr.work_day
		LEFT JOIN ods_fox_asset asset ON ml.mission_log_asset_id = asset.asset_id
	WHERE
		ml.mission_log_operator = 'assign'
		AND (ml.mission_log_create_at >= '{0}' AND ml.mission_log_create_at <= '{1}')
		AND ml.mission_group_name NOT IN ('PHCC','IVR','WIZ-IVR','cultivate')) a
	WHERE
		a.`是否计入分案本金` = 1)
GROUP BY
	uo.asset_group_name,
	uo.manager_user_name,
	uo.leader_user_name,
	she.dunner_id'''

In [9]:
# 业绩数据查询
df_cycle = pd.read_sql(query_sql_performance.format(str(start_time),str(end_time)),connection)
df_month = pd.read_sql(query_sql_performance.format(str(first_start_time),str(end_time)),connection)
# 上线明细及新老人标记
df2 = pd.read_sql(query_sql_online_new.format(str(start_time),str(end_time)),connection)
# 电话拨打数据
df3 = pd.read_sql(query_sql_call.format(str(start_time),str(end_time)),connection)
# 短信发送量
df4 = pd.read_sql(query_sql_sms.format(str(start_time),str(end_time)),connection)


In [ ]:
# 汇总上线日期 及 是否新人标志
df_online_new_sum = df2.groupby(['asset_group_name','manager_user_name','leader_user_name','user_id']).agg({
    "attendance_status":sum,
    "是否新人":lambda x : x.iloc[0]
}).reset_index().rename(columns = {
    'asset_group_name' : '业务组',
    'manager_user_name' : '主管',
    'leader_user_name' : '组长',
    'user_id' : '催员ID',
    "attendance_status" : '上线天数'
})

In [ ]:
# 合并上线天数及周期内业绩数据
merge_result_online_cycle = pd.merge(df_online_new_sum,df_cycle,on = ['业务组','主管','组长','催员ID'],how='left').rename(columns = {
    '新案分案个数' : '周期内新案分案个数',
     '分案个数' : '周期内分案个数',
     '新案分案本金' : '周期内新案分案本金',
     '总分案本金' : '周期内总分案本金',
     '总实收' : '周期内总实收',
     '新案D1回款本金' : '周期内新案D1回款本金',
     '新案D1回款的分案本金' : '周期内新案D1回款的分案本金',
     '新案回款本金' : '周期内新案回款本金',
     '回款本金' : '周期内回款本金'
})

In [ ]:
# 合并月累计业绩数据
df_month_cleaned = df_month.drop(['新案分案个数','分案个数', '总实收','新案D1回款本金','新案D1回款的分案本金'],axis=1).rename(columns = {
    '新案分案本金':'月累计新案分案本金',
      '总分案本金' : '月累计总分案本金',
     '新案回款本金' : '月累计新案回款本金',
     '回款本金' : '月累计回款本金'
})

# 业绩数据合并完成
performance_data = pd.merge(merge_result_online_cycle,df_month_cleaned,on = ['业务组','主管','组长','催员ID'],how='left')

In [ ]:
# 合并业绩数据与过程数据
performance_call = pd.merge(performance_data,df3,on = ['业务组','主管','组长','催员ID'],how='left')
performance_call_sms_result = pd.merge(performance_call,df4,on = ['业务组','主管','组长','催员ID'],how='left').fillna(0)

In [ ]:
# 正式主管替换储备主管
performance_call_sms_result['主管'] = performance_call_sms_result['主管'].apply(lambda x: 'Allen(Account)' if x == 'Evan（account）' else x)

In [ ]:
# 业务组 主管 是否新人分组聚合数据
group_sp_data = performance_call_sms_result.groupby(['业务组','主管','是否新人']).agg(
    总人数=('催员ID','count'),
    组数=('组长','nunique'),
    周期内总上线天数 = ('上线天数','sum'),
    周期内新案分案个数 = ('周期内新案分案个数','sum'),
    周期内分案个数 = ('周期内分案个数','sum'),
    周期内新案分案本金 = ('周期内新案分案本金','sum'),
    周期内总分案本金 = ('周期内总分案本金','sum'),
    周期内总实收 = ('周期内总实收','sum'),
    周期内新案D1回款本金 = ('周期内新案D1回款本金','sum'),
    月累计新案总催回本金 = ('月累计新案回款本金','sum'),
    月累计总新案分案本金 = ('月累计新案分案本金','sum'),
    月累计总催回本金 = ('月累计回款本金','sum'),
    月累计总分案本金 = ('月累计总分案本金','sum'),
    周期内总外呼次数 = ('外呼次数','sum'),
    周期内总通话时长 = ('接通时长','sum'),
    周期内总接通次数 = ('接通次数','sum'),
    周期内总短信发送数 = ('短信发送数量','sum'),
    周期内分案三方总外呼号码数 = ('三方总外呼号码数量','sum'),
    周期内分案已拨打总债务人 = ('总外呼债务人','sum'),
    周期内本人接通总债务人 = ('本人接通总债务人','sum'),
    周期内三方接通总债务人 = ('三方接通总债务人','sum'),
    周期内接通总债务人 = ('接通总债务人','sum')
).reset_index()

# 业务组 是否新人 分组聚合数据
group_data = performance_call_sms_result.groupby(['业务组','是否新人']).agg(
    总人数=('催员ID','count'),
    组数=('组长','nunique'),
    周期内总上线天数 = ('上线天数','sum'),
    周期内新案分案个数 = ('周期内新案分案个数','sum'),
    周期内分案个数 = ('周期内分案个数','sum'),
    周期内新案分案本金 = ('周期内新案分案本金','sum'),
    周期内总分案本金 = ('周期内总分案本金','sum'),
    周期内总实收 = ('周期内总实收','sum'),
    周期内新案D1回款本金 = ('周期内新案D1回款本金','sum'),
    月累计新案总催回本金 = ('月累计新案回款本金','sum'),
    月累计总新案分案本金 = ('月累计新案分案本金','sum'),
    月累计总催回本金 = ('月累计回款本金','sum'),
    月累计总分案本金 = ('月累计总分案本金','sum'),
    周期内总外呼次数 = ('外呼次数','sum'),
    周期内总通话时长 = ('接通时长','sum'),
    周期内总接通次数 = ('接通次数','sum'),
    周期内总短信发送数 = ('短信发送数量','sum'),
    周期内分案三方总外呼号码数 = ('三方总外呼号码数量','sum'),
    周期内分案已拨打总债务人 = ('总外呼债务人','sum'),
    周期内本人接通总债务人 = ('本人接通总债务人','sum'),
    周期内三方接通总债务人 = ('三方接通总债务人','sum'),
    周期内接通总债务人 = ('接通总债务人','sum')
).reset_index()

In [ ]:
# 最终数据计算 (业务组 主管 是否新人)
group_sp_data['人数/组数'] = group_sp_data['总人数'].astype(str) + '/' + group_sp_data['组数'].astype(str)

group_sp_data['是否新人'] = group_sp_data['是否新人'].replace({0:'NO',1:'YES'})

group_sp_data['日人均新案个数'] = (group_sp_data['周期内新案分案个数'] / group_sp_data['周期内总上线天数']).round(0)

group_sp_data['日人均新案本金'] = (group_sp_data['周期内新案分案本金'] / group_sp_data['周期内总上线天数']).round(0)

group_sp_data['日人均分案个数'] = (group_sp_data['周期内分案个数'] / group_sp_data['周期内总上线天数']).round(0)

group_sp_data['日人均总分案本金'] = (group_sp_data['周期内总分案本金'] / group_sp_data['周期内总上线天数']).round(0)

group_sp_data['日人均实收'] = (group_sp_data['周期内总实收'] / group_sp_data['周期内总上线天数']).round(0)

group_sp_data['新案催回率'] = (group_sp_data['月累计新案总催回本金'] / group_sp_data['月累计总新案分案本金']).round(4)

group_sp_data['总催回率'] = (group_sp_data['月累计总催回本金'] / group_sp_data['月累计总分案本金']).round(4)

group_sp_data['新案D1催回率'] = (group_sp_data['周期内新案D1回款本金'] / group_sp_data['周期内新案分案本金']).round(4)

group_sp_data['日人均外呼次数'] = (group_sp_data['周期内总外呼次数'] / group_sp_data['周期内总上线天数']).round(0)

group_sp_data['日人均通时'] = (group_sp_data['周期内总通话时长'] / group_sp_data['周期内总上线天数']).round(0)

group_sp_data['日人均通次'] = (group_sp_data['周期内总接通次数'] / group_sp_data['周期内总上线天数']).round(0)

group_sp_data['日人均短信发送量'] = (group_sp_data['周期内总短信发送数'] / group_sp_data['周期内总上线天数']).round(0)

group_sp_data['接通率'] = (group_sp_data['周期内总接通次数'] / group_sp_data['周期内总外呼次数']).round(4)

group_sp_data['单个债务人三方联系个数'] = (group_sp_data['周期内分案三方总外呼号码数'] / group_sp_data['周期内分案已拨打总债务人']).round(1)

group_sp_data['案件覆盖率'] = (group_sp_data['周期内分案已拨打总债务人'] / group_sp_data['周期内分案个数']).round(4)

group_sp_data['本人触达率'] = (group_sp_data['周期内本人接通总债务人'] / group_sp_data['周期内分案个数']).round(4)

group_sp_data['三方触达率'] = (group_sp_data['周期内三方接通总债务人'] / group_sp_data['周期内分案个数']).round(4)

group_sp_data['案件触达率'] = (group_sp_data['周期内接通总债务人'] / group_sp_data['周期内分案个数']).round(4)

In [ ]:
# 最终数据计算 (业务组 是否新人)
group_data['人数/组数'] = group_data['总人数'].astype(str) + '/' + group_data['组数'].astype(str)

group_data['是否新人'] = group_data['是否新人'].replace({0:'NO',1:'YES'})

group_data['日人均新案个数'] = (group_data['周期内新案分案个数'] / group_data['周期内总上线天数']).round(0)

group_data['日人均新案本金'] = (group_data['周期内新案分案本金'] / group_data['周期内总上线天数']).round(0)

group_data['日人均分案个数'] = (group_data['周期内分案个数'] / group_data['周期内总上线天数']).round(0)

group_data['日人均总分案本金'] = (group_data['周期内总分案本金'] / group_data['周期内总上线天数']).round(0)

group_data['日人均实收'] = (group_data['周期内总实收'] / group_data['周期内总上线天数']).round(0)

group_data['新案催回率'] = (group_data['月累计新案总催回本金'] / group_data['月累计总新案分案本金']).round(4)

group_data['总催回率'] = (group_data['月累计总催回本金'] / group_data['月累计总分案本金']).round(4)

group_data['新案D1催回率'] = (group_data['周期内新案D1回款本金'] / group_data['周期内新案分案本金']).round(4)

group_data['日人均外呼次数'] = (group_data['周期内总外呼次数'] / group_data['周期内总上线天数']).round(0)

group_data['日人均通时'] = (group_data['周期内总通话时长'] / group_data['周期内总上线天数']).round(0)

group_data['日人均通次'] = (group_data['周期内总接通次数'] / group_data['周期内总上线天数']).round(0)

group_data['日人均短信发送量'] = (group_data['周期内总短信发送数'] / group_data['周期内总上线天数']).round(0)

group_data['接通率'] = (group_data['周期内总接通次数'] / group_data['周期内总外呼次数']).round(4)

group_data['单个债务人三方联系个数'] = (group_data['周期内分案三方总外呼号码数'] / group_data['周期内分案已拨打总债务人']).round(1)

group_data['案件覆盖率'] = (group_data['周期内分案已拨打总债务人'] / group_data['周期内分案个数']).round(4)

group_data['本人触达率'] = (group_data['周期内本人接通总债务人'] / group_data['周期内分案个数']).round(4)

group_data['三方触达率'] = (group_data['周期内三方接通总债务人'] / group_data['周期内分案个数']).round(4)

group_data['案件触达率'] = (group_data['周期内接通总债务人'] / group_data['周期内分案个数']).round(4)

In [ ]:
# 剔除业务组为空的数据
result_sp_data = group_sp_data[group_sp_data['业务组'] != ''][['业务组', '主管','人数/组数','是否新人','日人均新案个数','日人均新案本金','日人均分案个数','日人均总分案本金','日人均实收','新案催回率','总催回率','新案D1催回率','日人均外呼次数','日人均通时','日人均通次'
                                                            ,'日人均短信发送量','接通率','单个债务人三方联系个数','案件覆盖率','本人触达率','三方触达率','案件触达率']]

In [ ]:
# 设置按业务组汇总的数据的主管架构为空，并剔除业务组为空的数据
group_data['主管'] = group_data['主管'] = ''
result_group_data = group_data[group_sp_data['业务组'] != ''][['业务组', '主管','人数/组数','是否新人','日人均新案个数','日人均新案本金','日人均分案个数','日人均总分案本金','日人均实收','新案催回率','总催回率','新案D1催回率','日人均外呼次数','日人均通时','日人均通次'
                                                            ,'日人均短信发送量','接通率','单个债务人三方联系个数','案件覆盖率','本人触达率','三方触达率','案件触达率']]

In [ ]:
# 合并主管数据和业务组数据
relust_df_data = pd.concat([result_sp_data,result_group_data],ignore_index=True)

In [ ]:
# 排序后导出excel
# relust_df_data.sort_values(by=['是否新人','业务组','主管'],ascending=[True,True,False]).to_excel('./菲律宾电催周报（月报过程数据）.xlsx',index=False)
sorted_result = relust_df_data.sort_values(by=['是否新人','业务组','主管'],ascending=[True,True,False])
sorted_result.to_excel('./菲律宾电催周报过程数据(0711-0717).xlsx',index=False)